In [ ]:
# default_exp text

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from fastcore.basics import listify
import faiss
from typing import List

In [ ]:
#export
STRING_FUNCS = ["capitalize", "count", "isalnum", "isalpha", "isascii", "isdecimal", "isdigit", "isidentifier", "islower", 
                "isnumeric", "isspace", "istitle", "isupper", "lower", "lstrip", "replace", "rstrip", "split", "strip", "upper"]

In [ ]:
#export
# TODO: add convert so that i automatically converts a string/np array/dataframe/tensor into a list
def text_to_vector(text_form, embedder):
    embeddings = embedder.encode(text_form, convert_to_tensor=True)
    return embeddings.cpu().detach().numpy()

In [ ]:
#export
def create_index(vectors):
    faiss_index = faiss.IndexFlatL2(len(vectors[0]))
    faiss_index.add(vectors)
    
#     print(faiss_index.ntotal)

    return faiss_index

In [ ]:
#export
def query_index(text, embedder, target_list, index, with_distance=False, k=10):
    embedding = embedder.encode([text])
    distances, indices = index.search(embedding, k)
    if with_distance:
        return [(target_list[index], distances[0][i]) for i, index in enumerate(indices[0])]
    return [target_list[i] for i in indices[0]]

In [ ]:
def hypernyms(ngram: str):
    """
    parent in an ontology
    """
    if "WORDNET":
        pass
    if "CORPUS_FILE":
        hearst_patterns("zxz")


def hearst_patterns(query: str) -> List[str]:
    """
    animals like a, b, c
    """
    pass

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted aws_utils.ipynb.
Converted index.ipynb.
Converted load.ipynb.
Converted roam_utils.ipynb.
Converted semanticscholar_api.ipynb.
Converted spec.ipynb.
Converted text.ipynb.
Converted utils.ipynb.
